In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import re
import os
import csv
from langdetect.lang_detect_exception import LangDetectException

In [2]:
def print_list(l):
    lenght=len(l)
    if lenght==0:
        return ""
    if lenght==1:
        return l[0]
    to_print=""
    if lenght>1:
        for i in range(lenght-1):
            to_print+=l[i]+", "
        to_print+=l[lenght-1]
    return to_print

In [3]:
class bookTitle():
    def name():
        return "bookTitle"
    def parse(soup):
        bookTitle = soup.find_all('h1')[0].contents[0]
        bookTitle = " ".join(bookTitle.split())
        return (bookTitle)

class bookSeries():
    def name():
        return "bookSeries"
    def parse(soup):
        bookSeries=""
        bookSeries=soup.find('h2',id="bookSeries").text.strip()[1:-1]
        return bookSeries

class bookAuthors():
    def name():
        return "bookAuthors"
    def parse(soup):
        bookAuthors=[]
        for element in soup.find_all("span",itemprop="name"):
            bookAuthors.append(element.text.strip())
        return print_list(bookAuthors)

class ratingValue():
    def name():
        return "ratingValue"
    def parse(soup):
        ratingValue = soup.find_all('span',itemprop="ratingValue")[0].contents[0].split('\n')[1].strip() 
        return ratingValue

class ratingCount():
    def name():
        return "ratingCount"  
    def parse(soup):
        return str(soup.find("meta",itemprop="ratingCount").get("content"))

class reviewCount():
    def name():
        return "reviewCount"
    def parse(soup):
        return str(soup.find("meta",itemprop="reviewCount").get("content"))


class Plot():
    def name():
        return "Plot"
    def parse(soup):
        def headingToRemove(Plot): 
            to_check=Plot.find("i")
            if to_check:
                forbidden_strings=["isbn","edition","librarian's note"]
                for string in forbidden_strings:
                    if string in to_check.text.lower():
                        Plot.find("i").decompose()

        Plot=soup.find("div", id="descriptionContainer").find_all("span")

        if len(Plot)==2:
            Plot=Plot[1]
            headingToRemove(Plot)
            Plot=Plot.text
            Plot=" ".join(Plot.split())
            Plot=Plot.replace("\\","")
        elif len(Plot)==1: 
            Plot=Plot[0]
            headingToRemove(Plot)
            Plot=Plot.text
            Plot=" ".join(Plot.split())
            Plot=Plot.replace("\\","")
        else:
            Plot=""
        return Plot


class NumberOfPages():
    def name():
        return "NumberOfPages"
    def parse(soup):
        N_pages=soup.find_all('span', itemprop="numberOfPages")
        if N_pages:
            return N_pages[0].contents[0].replace('\n', '').strip().split()[0]
        return ""

class Publishing_Date():
    def name():
        return "Publishing_Date"
    def parse(soup):
        elements = [e for e in soup.find_all("div", class_="row") if re.match(r'Published',e.text.strip())]
        #We first try to get the "first published date"
        if elements:
            date=re.findall(r'(?<=\(first published )(.*?)(?=\))',elements[0].text)
        else:
            return ""
        if date:
            return date[0]
        #We now see if there is a publishing date (but not a first publishing one).
        date=" ".join(elements[0].text.split()).split()
        #Handling the issue that not always the date is in the same format 
        if date[1]!="by":
            Publishing_Date=date[1]
            if len(date)>2 and date[2]!="by":
                Publishing_Date+=" "+date[2]
                if len(date)>3 and date[3]!="by":
                    Publishing_Date+=" "+date[3]
            return Publishing_Date
        else:
            return ""

        
class Characters():
    def name ():
        return "Characters"
    def parse(soup):
        Characters=soup.find_all("a",{'href': re.compile(r'^/characters/')})
        characters=[]
        for item in Characters:
            characters.append(" ".join(item.text.split()))
        return print_list(characters)

class Setting():
    def name():
        return "Setting"
    def parse(soup):
        Setting_temp=soup.find_all("div",class_="infoBoxRowItem")
        Setting=[]
        temp=[]
        Setting_places = []
        for element in Setting_temp:
            if element.find("a",{'href': re.compile(r'^/places/')}):
                Setting_places=element
        if Setting_places:
            temp=Setting_places.find_all()
        else:
            Setting=[]
        for element in temp:
            if element.name=="a":
                to_insert=element.text.split()
                Setting.append(" ".join(to_insert))
            if element.name=="span":
                to_add=element.text.split()
                Setting[-1]+=" "+(" ".join(to_add))
        #This is only a vert long workaround but seems to work
        for i in range(len(Setting)):
            Setting[i]=Setting[i].replace("…more","").replace("…less","").strip()
        Setting=list(dict.fromkeys([x for x in Setting if x]))
        return print_list(Setting)


class Url():
    def name():
        return "Url"
    def parse(soup):
        return re.findall(r'(?<=link href=")(.*?)(?=")',str(soup))[0]

In [11]:
#TEST
name="/Users/domenicospoto/Desktop/Sapienza/MScDataScience/AlgorithmicMethodsofDataMiningandLaboratory/HW3/ADM3/"+str(3)+"/"+str(3)+".html"
with open(name,"r",encoding="utf-8") as file:
    #name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(3)+"\\"+str(3)+".html"
    soup=BeautifulSoup(file,features="html.parser")
Setting.parse(soup)

'Ajiro (Japan), Tokyo (Japan)'

In [ ]:
functions=[bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberOfPages,Publishing_Date,\
           Characters,Setting,Url]

header = ""
for fun in functions[:-1]:
    header += fun.name()+"\t"
header +=functions[-1].name()
count = 1 #for the articles

for i in range(1,301):
    for j in range(1,101):
        name="/Users/domenicospoto/Desktop/Sapienza/MScDataScience/AlgorithmicMethodsofDataMiningandLaboratory/HW3/ADM3/"+str(3)+"/"+str(3)+".html"
        #name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(i)+"\\"+str(j)+".html"
        try:
            with open(name,"r",encoding="utf-8") as file:
                if os.path.getsize(name)<100000: #ignoring "bad pages"
                    continue
                soup=BeautifulSoup(file,features="html.parser")
                second_line = ""
                for fun in functions[:-1]:
                    try:
                        second_line += fun.parse(soup)+"\t"
                    except:
                        print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i)
                        second_line += "" + "\t"
                try:
                    second_line += functions[-1].parse(soup)  
                except:
                    print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i) 
                    second_line += "" + "\t"
            name="C:\\Users\\Stefania\\ADM_HW3\\articles\\articles_"+str(count)+".tsv"
            with open(name,"w",encoding="utf-8") as file:
                file.write(header)
                file.write("\n")
                file.write(second_line)
            count +=1
        except FileNotFoundError:
            continue

In [4]:
def custom_detect(string):
    try:
        return detect(string)
    except LangDetectException:
        return "BADLANGUAGE"

In [6]:
count = []
for i in range(1,29226):
    with open('articles/articles_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
        temp = csv.DictReader(file, delimiter = '\t')
        for row in temp:
            #print('file' + str(i))
            if custom_detect(row['Plot']) != 'en':
                count.append(i)
                print(i)

for i in count:
    os.remove('articles/articles_' + str(i) +'.tsv')

294
425
494
680
684
731
749
789
823
852
872
976
1030
1033
1039
1052
1166
1249
1281
1405
1419
1496
1501
1530
1645
1696
1734
1745
1762
1764
1766
1792
1864
1940
1942
1995
2011
2087
2099
2123
2222
2237
2321
2338
2364
2382
2450
2540
2574
2616
2639
2643
2646
2650
2652
2663
2671
2691
2708
2767
2783
2827
2847
2877
2883
2984
3050
3066
3086
3088
3123
3183
3200
3253
3261
3370
3371
3380
3427
3432
3477
3569
3574
3619
3631
3640
3656
3711
3720
3792
3815
3830
3870
3919
3936
3948
3966
3978
3990
3999
4016
4031
4042
4079
4095
4098
4128
4153
4164
4169
4178
4180
4211
4240
4336
4343
4362
4363
4368
4436
4455
4467
4494
4528
4565
4579
4636
4673
4744
4786
4790
4810
4828
4834
4869
4892
4906
4911
4913
4926
4941
4952
4975
5004
5020
5034
5041
5054
5114
5132
5146
5206
5230
5243
5264
5318
5320
5321
5335
5351
5381
5457
5498
5506
5507
5512
5527
5575
5585
5593
5599
5602
5616
5620
5663
5694
5725
5732
5766
5810
5876
5896
5905
5927
5935
5941
5953
5979
5986
5994
6012
6015
6020
6023
6034
6052
6054
6082
6087
6090
6134
6149
61

21656
21661
21667
21670
21673
21679
21681
21702
21706
21737
21746
21751
21753
21760
21766
21772
21793
21817
21823
21827
21838
21844
21859
21868
21874
21877
21880
21881
21883
21887
21889
21893
21898
21916
21930
21950
21956
21957
21960
21968
21970
21972
21993
21999
22003
22015
22016
22023
22046
22060
22072
22073
22075
22094
22097
22103
22109
22111
22116
22121
22125
22140
22153
22168
22176
22180
22183
22184
22186
22212
22213
22216
22221
22227
22233
22240
22245
22250
22257
22268
22270
22273
22276
22283
22284
22289
22301
22306
22313
22325
22326
22331
22340
22344
22359
22360
22363
22364
22371
22382
22389
22394
22397
22399
22404
22411
22414
22419
22420
22430
22432
22441
22449
22460
22463
22470
22471
22472
22478
22479
22481
22482
22489
22502
22506
22507
22510
22518
22522
22523
22536
22541
22552
22564
22571
22572
22574
22579
22582
22588
22596
22599
22603
22608
22613
22614
22619
22644
22647
22660
22662
22675
22678
22684
22690
22705
22710
22712
22714
22725
22728
22729
22730
22741
22747
22748
2275

In [8]:
counter = 1
for i in range(1, 29226):
    try:
        os.rename('articles/articles_' + str(i) +'.tsv', 'articles/article_' + str(counter) +'.tsv')
        counter += 1
    except FileNotFoundError:
        continue